# Imports

In [ ]:
from ecg_plotting import *
import IPython
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, initializers

# Data

In [ ]:
# download datasets

from dataset_downloader import download_dataset
import datasets.ptb_xl.data_handling as ptb_xl_dh

download_dataset('backgrounds')
download_dataset('ptb_xl')

## Processing and Labeling

### Assign labels

In [ ]:
from datasets.ptb_xl.data_handling import get_ecg_array

num_samples = 5000
X, Y_label = get_ecg_array(sampling_rate=500, max_samples=num_samples)
X = np.reshape(X[:,452:-452,:],(-1, 4096,12 ,1))
Y = np.zeros((num_samples))

danger_list = ["IMI", "ASMI", "ILMI", "AMI", "LMI", "IPLMI", "IPMI", "PMI"]
for k in range(num_samples):
    if len(set(Y_label['scp_codes'].iloc[k].keys()) & set(danger_list)) > 0:
        Y[k] = 1

### Split into train/test

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

### Check label distribution

#### Train set

In [ ]:
train_num_total = Y_train.shape[0]
train_num_mi = np.sum(Y_train)
train_mi_ratio = train_num_mi / train_num_total
print('MI/non-MI proportion in train set: %.2f' % (100 * train_mi_ratio) + '%')

fig, ax = plt.subplots()
plot = plt.bar(x=[0, 1], height=[train_num_total - train_num_mi, train_num_mi], color=['b', 'r'], tick_label=['No MI', 'MI'])

### Test set

In [ ]:
test_num_total = Y_test.shape[0]
test_num_mi = np.sum(Y_test)
test_mi_ratio = test_num_mi / test_num_total
print('MI/non-MI proportion in test set: %.2f' % (100.0 * test_mi_ratio) + '%')

fig, ax = plt.subplots()
plot = plt.bar(x=[0, 1], height=[test_num_total - test_num_mi, test_num_mi], color=['b', 'r'], tick_label=['No MI', 'MI'])

# Network Definition

### Initializers

In [ ]:
weight_init = initializers.HeNormal()
bias_init = initializers.Zeros()
init = { 'kernel_initializer': weight_init, 'bias_initializer': bias_init }

### Residual block

In [ ]:
class Residual(layers.Layer):
    def __init__(self, last_num_filters, **kwargs):
        super(Residual, self).__init__(**kwargs)
        self.last_num_filters = last_num_filters

        self.layer_1 = layers.Conv2D(last_num_filters + 64, (16, 1), activation=None, padding='same', **init)
        self.layer_2 = layers.MaxPooling2D((2, 1)) # CHECK BECAUSE PAPER DOESN'T MENTION PRECISELY
        self.layer_3 = layers.BatchNormalization(axis=[1,2]) #? axis = [1,2] to normalize the axis=0 (over the batch)
        self.layer_4 = layers.Activation(activation='relu')
        self.layer_5 = layers.Dropout(dropout_p) # To prevent overfit
        self.layer_6 = layers.Conv2D(last_num_filters + 64, (16, 1), activation=None, padding='same', **init)
        self.layer_7 = layers.MaxPooling2D((2, 1)) # CHECK BECAUSE PAPER DOESN'T MENTION PRECISELY
        self.layer_8 = layers.BatchNormalization(axis=[1,2]) #? axis = [1,2] to normalize the axis=0 (over the batch)
        self.layer_9 = layers.Activation(activation='relu')
        self.layer_10 = layers.Dropout(dropout_p) # To prevent overfit

        self.layer_11 = layers.Conv2D(last_num_filters + 64, (1, 1), activation=None, padding='same', **init)
        self.layer_12 = layers.MaxPooling2D((4, 1)) # CHECK BECAUSE PAPER DOESN'T MENTION PRECISELY
        self.layer_13 = layers.BatchNormalization(axis=[1,2]) #? axis = [1,2] to normalize the axis=0 (over the batch)
        self.layer_14 = layers.Activation(activation='relu')
        self.layer_15 = layers.Dropout(dropout_p) # To prevent overfit

    def call(self, x):
        # the residual block using Keras functional API
        x_backup = x
        last_num_filters = self.last_num_filters

        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)
        x = self.layer_5(x)
        x = self.layer_6(x)
        x = self.layer_7(x)
        x = self.layer_8(x)
        x = self.layer_9(x)
        x = self.layer_10(x)

        x_backup = self.layer_11(x_backup)
        x_backup = self.layer_12(x_backup)
        x_backup = self.layer_13(x_backup)
        x_backup = self.layer_14(x_backup)
        x_backup = self.layer_15(x_backup)
        
        x = layers.Add()([x,x_backup])

        return x

### Network

In [ ]:
dropout_p = 0.1

model = models.Sequential()

model.add(layers.Conv2D(64, (16, 1), activation=None, input_shape=(4096, 12, 1), **init))
model.add(layers.BatchNormalization(axis=[1, 2])) #? axis = [1,2] to normalize the axis=0 (over the batch)
model.add(layers.Activation(activation='relu'))
model.add(layers.Dropout(dropout_p)) # to prevent overfit

last_num_filters = 64
model.add(Residual(last_num_filters))
last_num_filters += 64
model.add(Residual(last_num_filters))
last_num_filters += 64
model.add(Residual(last_num_filters))
last_num_filters += 64
model.add(Residual(last_num_filters))
last_num_filters += 64

model.add(layers.Flatten())

model.add(layers.Dense(1, activation='sigmoid', **init))

model.summary()

## Compile

### Add metrics

In [ ]:
# Credits to https://datascience.stackexchange.com/a/45166
auc_metric = tf.keras.metrics.AUC(num_thresholds=200)

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

### Compile model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc_metric, f1_m,precision_m, recall_m])

## Train

In [ ]:
# Check whether we're training on a GPU or not
tf.test.is_gpu_available()

In [ ]:
model.fit(X_train, Y_train, epochs=40, batch_size=16, validation_data=(X_test, Y_test))